<a href="https://colab.research.google.com/github/0yacinek0/nlp/blob/practice/%5Bgithub_nlp%5D_2_%ED%96%A5%EC%83%81%EB%90%9C_%EA%B0%90%EC%A0%95_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 구글 드라이브 마운팅 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd # 현재 경로 확인 

'/content'

In [ ]:
%cd /content/drive/MyDrive/논문/sentimentP/dataset

/content/drive/MyDrive/논문/sentimentP/dataset


In [ ]:
pwd

'/content/drive/MyDrive/논문/sentimentP/dataset'

In [ ]:
ls

Mecab-ko-for-Google-Colab/  ratings_train.txt  train_data.csv
ratings_test.txt            test_data.csv


### 필요한 패키지와 랜덤시드 설정

In [ ]:
# !pip install torchtext==0.10.1 
!pip install torch==1.8.0 torchtext==0.9.0
"""
아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.

torchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?
pip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.

https://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils

"""

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.


'\n아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.\n\ntorchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?\npip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.\n\nhttps://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils\n\n'

In [ ]:
import torch
from torchtext import data

SEED = 1027

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# 전처리
> Filed 지정 / KoNLPy의 은전한닢(Mecab) tokenizer 이용

In [ ]:
# 코랩 내 다운로드 https://wikidocs.net/94600
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/논문/sentimentP/dataset/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 37.0 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-29 16:45:06--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downl

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
import torchtext
from torchtext import data
from torchtext.legacy import data

TEXT = torchtext.legacy.data.Field(tokenize=mecab.morphs)
LABEL = torchtext.legacy.data.LabelField(dtype = torch.float)

In [ ]:
torch.__version__
torchtext.__version__

'0.9.0'

### torchtext 내 내장된 데이터셋 이용하는게 아니므로 컬럼별 해당 Field 지정

In [ ]:
fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}
# dictionary 형식; {csv 컬럼명 : (데이터 컬럼명, Field이름)}

In [ ]:
train_data, test_data = data.TabularDataset.splits(
    path = '.',
    train = 'train_data.csv',
    test = 'test_data.csv',
    format = 'csv',
    fields = fields
)

In [ ]:
vars(train_data[0]), vars(train_data[1])

({'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'],
  'label': '0'},
 {'text': ['흠',
   '.',
   '..',
   '포스터',
   '보고',
   '초딩',
   '영화',
   '줄',
   '.',
   '...',
   '오버',
   '연기',
   '조차',
   '가볍',
   '지',
   '않',
   '구나'],
  'label': '1'})

In [ ]:
print(f'훈련 데이터 수: {len(train_data)}')
print(f'테스트 데이터 수: {len(test_data)}')

훈련 데이터 수: 149995
테스트 데이터 수: 49997


### 검증데이터 별도 생성

In [ ]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [ ]:
print(f'훈련데이터 수 : {len(train_data)}')
print(f'검증 데이터 수 : {len(valid_data)}')
print(f'테스트 데이터 수 : {len(test_data)}')

훈련데이터 수 : 104996
검증 데이터 수 : 44999
테스트 데이터 수 : 49997


# 단어 벡터는 전처리된 단어 벡터를 받아보기
> * 한글을 지원하는 fasttext.simple.300d 사용 <br>
> * 사전훈련된 단어집에 없는 단어는 0으로 처리하는 걸 방지하기 위해 unk_init = otrch.Tensor.normal_옵션을 줌

### 총 단어의 수 확인

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                 max_size=MAX_VOCAB_SIZE,
                 vectors = 'fasttext.simple.300d',
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [ ]:
len(TEXT.vocab)

25002

In [ ]:
LABEL.vocab.stoi

defaultdict(None, {'0': 0, '1': 1})

In [ ]:
vars(train_data.examples[0])

{'text': ['정신',
  '적',
  '인',
  '문제',
  '는',
  '반드시',
  '무조건',
  '치료',
  '로',
  '해결',
  '을',
  '해야',
  '한다',
  '!'],
 'label': '0'}

In [ ]:
for i in range(len(train_data)):
  if len(train_data.examples[i].text)==0:
     print(i)

### BuketIterator를 이용하여 데이터 생성자 만들기
* 데이터 생성자를 만드는데, 길이에 따라 정렬하도록 sort_within_batch = True옵션을 넣어줄 것을 원 튜토리얼에서 요구하나
* 한글 데이터에선 오류가 발생하여 sort_key=lambda x:len(x.text) 문장을 먼저 넣어줘야 오류없이 작동함

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    # sort_within_batch = True,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

### 패딩 제외 길이로 정렬된 (문장, 길이) 순의 데이터로 이루어져있음

In [ ]:
next(iter(train_iterator)).text

tensor([[   28,    47,   142,   376,   163,  1249,    97,    28,   260,   166,
           106,   187,   646,   612,    76,    47, 11192,  1073,  2445, 11052,
           219,  5255, 14654,   267,  2477,  3583,   131,   692,  1407,     3,
         22226,  8621,  6741,   131,   453,   926,   264,  7417,  3226,  4597,
          4249,    17,   861,    32, 12684,  1314, 11745,   539,    28,   101,
            16,   993,    47,   191,  3552,  1071,    41,   227,    67,  1484,
          3431,  1325,   257,   114],
        [  199,   189,    36,   237,    98,  4370,     2,     4,   161,    10,
            11,   167,    84,     5,   160,   111,    11,   534,    10,    16,
             4,    10,  2302,     2,  1349,    36,   490,     0,   214,     5,
          1151,  9809,   216,     5,   340,     9,    12,   925,     8, 10229,
            14,  1173,   194,   106,    40,  2021,     3,   208,     9,     0,
           643,    95,    42,  3364,   197,  3248,     2,   371,    11,     8,
            13

# 모델 생성
* multi-layered bi-directional LSTM 사용
* dropout 적용
* nn.utils.rnn.packed_padded_sequence 써서 패킹/언패킹 처리
<br>
* 최종 hidden size는 [num layers*num directions, batch size, hidden dim]임
* 구체적으로 [forward_layer_0, backward_layer_0, forward_layer_1, backward_layer1, ..., forward_layer_n, backward_layer n] 같이 출력.
* 꼭대기층의 hidden만 필요하므로 hidden[-2::]과 hidden[-1::]만 뽑아 concatenate할 예정

In [ ]:
import torch.nn as nn

In [ ]:
emb = nn.Embedding(3,5,padding_idx=1) 
test = torch.tensor([0, 1, 2])

In [ ]:
emb(test) # padding_idx에 해당하는 벡터는 0임

tensor([[ 0.5421,  0.3857,  0.9083, -1.1629,  0.2727],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1429, -0.0602, -0.0263, -0.8012, -1.6145]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
def print_shape(name, data):
  print(f'{name} has shape {data.shape}')

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers,
                          bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        # text = [sent_len, batch_size]
        #print_shape('text',text)
        embedded = self.dropout(self.embedding(text))
        # embedded = [sent_len, batch_size, emb_dim]
        #print_shape('embedded', embedded)
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #print_shape('packed_output', packed_output)
        #print_shape('hidden', hidden)
        #print_shape('cell', cell)
        
        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #print_shape('output', output)
        #print_shape('output_lengths', output_lengths)        
        
        # output = [sent_len, batch_size, hi_dim * num_directions]
        # output over padding tokens are zero tensors
        # hidden = [num_layers * num_directions, batch_size, hid_dim]
        # cell = [num_layers * num_directions, batch_size, hid_dim]
        
        # concat the final forward and backward hidden layers
        # and apply dropout
        
        #print_shape('hidden[-2,:,:]', hidden[-2,:,:])
        #print_shape('hidden[-1,:,:]', hidden[-1,:,:])
        #cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        #print_shape('cat', cat)

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        #print_shape('hidden', hidden)
        # hidden = [batch_size, hid_dim * num_directions]
        
        res = self.fc(hidden)
        #print_shape('res', res)
        return res

### 하이퍼파라미터 설정

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300 # fasttext dim과 동일하게 설정
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [ ]:
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
           N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

### 파라미터 갯수 세보기 
* 앞 모델과 비교하여 월등하게 많은 파라미터 수를 확인할 수 있음 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'이 모델은 {count_parameters(model):,} 개의 파라미터를 가지고 있다.')

이 모델은 10,220,857 개의 파라미터를 가지고 있다.


### 사전 학습된 fasttxt모델의 단어 벡터를 embedding 레이어에 복사하여 담기

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 300])


### Dimension 체크

In [ ]:
model.embedding.weight.data.shape

torch.Size([25002, 300])

### weight가 아니라 weight.data 에 덮어씌워야 한다는 걸 명심

In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings) # copy_ 메서드는 인수를 현재 모델의 웨이트에 복사함

tensor([[-1.1297e-01,  1.2156e+00,  6.8516e-01,  ..., -5.2034e-01,
         -1.6626e-01, -8.4676e-04],
        [ 9.0628e-01,  3.6853e-01, -6.4057e-01,  ...,  7.0754e-01,
          6.3230e-01,  3.6939e-01],
        [ 5.6857e-02, -5.1956e-02,  2.7326e-01,  ..., -6.9453e-02,
         -1.6064e-01, -9.8923e-02],
        ...,
        [ 1.6462e-01,  3.1504e-01, -2.1961e-01,  ...,  1.2704e+00,
          3.3490e-02,  6.4178e-01],
        [ 9.2339e-01,  6.3520e-01, -8.0703e-01,  ...,  4.7187e-02,
          9.4318e-01,  1.3437e+00],
        [-3.4884e-01,  1.5506e+00, -3.0697e-01,  ..., -9.4593e-01,
          5.8037e-01,  7.7458e-01]])

### unk와 pad는 수동으로 0벡터로 만들자.

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
UNK_IDX, PAD_IDX

(0, 1)

In [ ]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [ 0.1646,  0.3150, -0.2196,  ...,  1.2704,  0.0335,  0.6418],
        [ 0.9234,  0.6352, -0.8070,  ...,  0.0472,  0.9432,  1.3437],
        [-0.3488,  1.5506, -0.3070,  ..., -0.9459,  0.5804,  0.7746]])


### pad는 pad_idx 옵션 때문에 훈련 내내 0으로 남아있지만 unk는 학습될 것!

# 모델 학습
> adam으로 학습 

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

### 손실 함수는 binary cross entropy with logits로 하자. 이 함수는 임의의 실수를 입력으로 받아서 sigmoid 함수를 취해 0과 1 사이의 값으로 변환한 뒤 label과의 binary cross entropy를 계산

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

### 평가를 위해 임의의 실수를 0과 1 두 정수 중 하나로 변환하는 함수

In [ ]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

### 훈련 함수. 현재 batch.text 는 (토큰들,문장 길이) 로 구성되어 있으니 분리

In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        #print_shape('predictions',predictions)
        
        loss = criterion(predictions, batch.label)
        #print_shape('loss',loss)
        
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### 평가를 위한 함수는 그래디언트 업데이트를 하지 않아야 하므로 with torch.no_grad(): 구문으로 감싸도록 한다. 또한 드랍아웃을 평가 때는 적용하지 않아야 하므로 model.eval() 을 넣어주어야 함

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)#.squeeze(0))
            acc = binary_accuracy(predictions, batch.label)#.squeeze(0))

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### 에폭마다 걸린 훈련시간을 측정하는 함수 

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### 훈련 

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

ValueError: ignored

### 테스트셋의 결과

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

FileNotFoundError: ignored

### 더 훈련시켜보기

In [ ]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
        
    print(f'Epoch: {epoch+6:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

ValueError: ignored

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

FileNotFoundError: ignored

# 사용자 데이터 사용
영화 평가 데이터 직접 넣어보자.

다음 기능을 하는 predict_sentiment 함수를 만들자.

sets the model to evaluation mode
tokenizes the sentence, i.e. splits it from a raw string into a list of tokens
indexes the tokens by converting them into their integer representation from our vocabulary
gets the length of our sequence
converts the indexes, which are a Python list into a PyTorch tensor
add a batch dimension by unsqueezeing
converts the length into a tensor
squashes the output prediction from a real number between 0 and 1 with the sigmoid function
converts the tensor holding a single value into an integer with the item() method

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok for tok in mecab.morphs(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1) # 배치 
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "이 영화 진짜 재밌었다!!")

0.5172811150550842

In [ ]:
predict_sentiment(model, "영화관에서 이걸 본 내가 바보다. 내 돈 돌려줘!!!")

0.5100839138031006